<a href="https://colab.research.google.com/github/xtructt/DC_DS_projs/blob/master/Product_Revenue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import datetime

Read csv file into the data frame

In [ ]:
prices = pd.read_csv('/content/Data_FIle/prices.csv',parse_dates=True)
sales = pd.read_csv('/content/Data_FIle/sales.csv',parse_dates=True)

In [ ]:
prices


,product_id,old_price,new_price,updated_at
0,64,270000,239000,9/10/18 16:37
1,3954203,60000,64000,9/11/18 11:54
2,3954203,60500,57500,9/17/18 22:59
3,3954203,64000,60500,9/15/18 3:49
4,3954203,68800,60000,9/10/18 16:32
5,3998909,15500,16500,9/16/18 5:09
6,3998909,17000,15500,9/13/18 6:43
7,3998909,19000,17000,9/10/18 16:35
8,4085861,53500,52000,9/17/18 22:59
9,4085861,53500,67000,9/12/18 3:51


In [ ]:
sales

,product_id,ordered_at,quantity_ordered
0,3998909,9/18/18 17:51,1
1,3998909,9/18/18 12:52,1
2,3998909,9/18/18 11:33,1
3,3998909,9/18/18 18:47,1
4,3998909,9/18/18 17:36,1
...,...,...,...
170,4085861,9/13/18 23:29,1
171,64,9/11/18 19:32,1
172,64,9/14/18 20:10,1
173,64,9/12/18 16:00,1


Mearge price and sale dataframe to get the price of product

In [ ]:
merge_prices_sales = pd.merge(sales,prices,how='left',on='product_id')
merge_prices_sales['time_diff'] = (pd.to_datetime(merge_prices_sales['ordered_at'])- pd.to_datetime(merge_prices_sales['updated_at']))



As there are many price ranges for a product we need to narrow down to find what is the correct price that we should be use to calculate the revenue for each transaction:

1. To do it we will need to calculate the time different between the transaction time and the price's updated date.
2. Based on this time difference, group each transaction by it product_id,ordered_at and get the min time difference for that group.
This can reflect the nearest update time of the price of the product.
3. Join the df back to the mearge table using product_id, ordered_at and time_diff
4. Get the price from the mearge result to calculate the revenue.


In [ ]:
mergered_group = merge_prices_sales[['product_id','ordered_at','quantity_ordered','old_price','new_price','time_diff']]
mergered_group_min = mergered_group[['product_id','ordered_at','time_diff']].groupby(by=['product_id','ordered_at']).min()
mergered_group_final = pd.merge(mergered_group_min,mergered_group,how='inner',on=['product_id','ordered_at','time_diff'])
mergered_group_final['diff_in_sec'] = mergered_group_final['time_diff'].astype('timedelta64[s]')

Split the dataframe in to 2 seperate dataframe:
1. the transaction that using the old_price column.
2. The transaction that using the new price column.

In [ ]:
mergered_group_final['is_old_price'] = mergered_group_final['diff_in_sec']<0
mergered_group_final_old_price= mergered_group_final[mergered_group_final['is_old_price']]
mergered_group_final_new_price= mergered_group_final[mergered_group_final['is_old_price']==False]

In [ ]:
mergered_group_final_old_price = mergered_group_final_old_price[['product_id','old_price','quantity_ordered']]
mergered_group_final_old_price.columns= ['product_id','price','quantity_ordered']


In [ ]:
mergered_group_final_new_price = mergered_group_final_new_price[['product_id','new_price','quantity_ordered']]
mergered_group_final_new_price.columns= ['product_id','price','quantity_ordered']


Mearge 2 dataframe above into one final dataframe.
Then calculate the revenue by multiply quantity by price.
Group the data by product_id and price than sum up the revenue to get the revenue of each product by its price

In [ ]:
revenue_table = pd.concat([mergered_group_final_new_price,mergered_group_final_old_price])
revenue_table[' revenue'] = revenue_table['price']*revenue_table['quantity_ordered']
revenue_table.groupby(by=['product_id','price']).sum()

quantity_ordered   revenue
product_id price                             
64         239000                 4    956000
3954203    57500                  1     57500
           60500                 13    786500
3998909    15500                  3     46500
           16500                 14    231000
4085861    52000                 20   1040000
           53500                123   6580500